First I need to create an engine using my database credentials.

In [1]:
from yaml import safe_load as yaml_load
from sqlalchemy import create_engine

with open(".gitignore/credentials_for_marketplace.yml") as file:
    credentials  = yaml_load(file)
DATABASE_TYPE = credentials['DATABASE_TYPE']
DBAPI = credentials['DBAPI'] 
ENDPOINT = credentials['ENDPOINT']       
USER = credentials['DBUSER']
PASSWORD = credentials['DBPASSWORD']
PORT = credentials['PORT']
DATABASE = credentials['DATABASE']
engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}")
        
        

I retrieve the tables from the database and create a dictionary consisting of the table names and a pandas dataframe. 

In [2]:
from sqlalchemy import inspect
inspector = inspect(engine)
table_names = inspector.get_table_names()
print(table_names)

['products_2', 'products', 'images']


In [4]:
import pandas as pd
tables ={}
for table_name in table_names: 
    tables[table_name] = pd.read_sql_table(table_name, engine)

I will start by investigating and cleaning the products table. Since products and products_2 have the same structure, I will create a variable for the table key to improve reusablity of my code. 

In [5]:
table_key = "products"
tables[table_key].head(10)

,id,product_name,category,product_description,price,location,page_id,create_time
0,ac2140ae-f0d5-4fe7-ac08-df0f109fd734,"Second-Hand Sofas, Couches & Armchairs for Sal...",N/A,N/A,N/A,N/A,1426592234,2022-02-26
1,243809c0-9cfc-4486-ad12-3b7a16605ba9,"Mirror wall art | in Wokingham, Berkshire | Gu...","Home & Garden / Dining, Living Room Furniture ...","Mirror wall art. Posted by Nisha in Dining, Li...",£5.00,"Wokingham, Berkshire",1426704584,2022-02-26
2,1c58d3f9-8b93-47ea-9415-204fcc2a22e6,"Stainless Steel Food Steamer | in Inverness, H...",Home & Garden / Other Household Goods,Morphy Richard’s (model no 48755)Stainless ste...,£20.00,"Inverness, Highland",1426704579,2022-02-26
3,860673f1-57f6-47ba-8d2f-13f9e05b8f9a,"Sun loungers | in Skegness, Lincolnshire | Gum...",Home & Garden / Garden & Patio / Outdoor Setti...,I have 2 of these - collection only as I don’t...,£20.00,"Skegness, Lincolnshire",1426704576,2022-02-26
4,59948726-29be-4b35-ade5-bb2fd7331856,Coffee side table from Ammunition ammo box hai...,"Home & Garden / Dining, Living Room Furniture ...",Great reclaimed army ammunition box used as co...,£115.00,"Radstock, Somerset",1426704575,2022-02-26
5,16dbc860-696e-4cda-93f6-4dd4926573fb,Modern Shannon Sofa for sale at low cost | in ...,"Home & Garden / Dining, Living Room Furniture ...",New Design Shannon Corner sofa 5 Seater Avail...,£450.00,"Delph, Manchester",1426704570,2022-02-26
6,5707be4f-49d9-4feb-b2c8-aa0868405c65,"Spotlights | in Newent, Gloucestershire | Gumtree",Home & Garden / Other Household Goods,New in box not needed Spotlights . Posted by s...,£9.00,"Newent, Gloucestershire",1426704571,2022-02-26
7,d2244a7b-aef4-4495-aff7-0c55bf512d6e,💥💥White Wardrobe💥💥Brand new Wardrobe For sale ...,Home & Garden / Beds & Bedroom Furniture / War...,🌈🌈Call Dustin 0208-0049437\r🌈🌈Same Day Or any...,£419.00,"Huddersfield, West Yorkshire",1426704568,2022-02-26
8,485edc04-7bbc-430a-9430-f05a77f5a917,"Letter cage | in Newent, Gloucestershire | Gum...",Home & Garden / Other Household Goods,Selling for family member brand new too big fo...,£10.00,"Newent, Gloucestershire",1426704516,2022-02-26
9,7e538445-c75e-4802-8dc5-a9083a802278,"Double Pine Bed Frame | in Lytham St Annes, La...",Home & Garden / Beds & Bedroom Furniture / Dou...,Double Pine bed frame good condition. Reasonab...,£50.00,"Lytham St Annes, Lancashire",1426704510,2022-02-26


In [6]:
tables[table_key].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8091 entries, 0 to 8090
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id                   8091 non-null   object        
 1   product_name         8091 non-null   object        
 2   category             8091 non-null   object        
 3   product_description  8091 non-null   object        
 4   price                8091 non-null   object        
 5   location             8091 non-null   object        
 6   page_id              8091 non-null   int64         
 7   create_time          8091 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 505.8+ KB


From this, I can see that I need to remove the row with N/A for price. For the moment I will drop any row with N/A in any column. 

The use of emojis may cause difficulty later, but for the moment I will leave them in as it could provide interesting information. 

The price should be converted to integer.

The location may be able to be converted into an area code.

I then need to deal with duplicated values.


In [19]:
tables[table_key].dropna(inplace=True)

In [8]:
def filter_rows_by_values(df, col, values):
    return df[~df[col].isin(values)]

tables[table_key] = filter_rows_by_values(tables[table_key], tables[table_key].columns, ["N/A"])

In [9]:
tables[table_key].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8091 entries, 0 to 8090
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id                   8091 non-null   object        
 1   product_name         8091 non-null   object        
 2   category             7156 non-null   object        
 3   product_description  7156 non-null   object        
 4   price                7156 non-null   object        
 5   location             7156 non-null   object        
 6   page_id              8091 non-null   int64         
 7   create_time          8091 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 568.9+ KB


This leaves me with around 900 less entries, which for the moment is an acceptable loss. But it may also be interesting to analyse how missing infomation in listings effects sales.

I'll now deal with the prices.

In [14]:
tables[table_key]["price"] = tables[table_key]["price"].str.strip("£")
tables[table_key]["price"] = tables[table_key]["price"].str.replace(",","")
tables[table_key]["price"] = tables[table_key]["price"].astype("float64")


The next thing to work on is converting the location into an area code so I can map information. 

In [20]:
set[tables[table_key]["location"]]

set[1               Wokingham, Berkshire
2                Inverness, Highland
3             Skegness, Lincolnshire
4                 Radstock, Somerset
5                  Delph, Manchester
                    ...             
8085     Acocks Green, West Midlands
8086     Acocks Green, West Midlands
8088                 Montrose, Angus
8089    Carrickfergus, County Antrim
8090                   Poole, Dorset
Name: location, Length: 7156, dtype: object]

The locations are too specific to be useful so I'll extract the county/district.

In [31]:
def extract_county(location):
    county = location.split(",")[-1].strip()
    return county

tables[table_key]["county"] = tables[table_key]["location"].apply(extract_county)

In [32]:
set(tables[table_key]["county"])

{'Aberdeen',
 'Aberdeenshire',
 'Angus',
 'Argyll and Bute',
 'Bedfordshire',
 'Belfast',
 'Berkshire',
 'Blaenau Gwent',
 'Bridgend',
 'Bristol',
 'Buckinghamshire',
 'Caerphilly',
 'Cambridgeshire',
 'Cardiff',
 'Carmarthenshire',
 'Cheshire',
 'Clackmannanshire',
 'Cornwall',
 'County Antrim',
 'County Armagh',
 'County Down',
 'County Durham',
 'County Fermanagh',
 'County Londonderry',
 'County Tyrone',
 'Cumbria',
 'Derbyshire',
 'Devon',
 'Dorset',
 'Dumfries and Galloway',
 'Dundee',
 'East Ayrshire',
 'East Dunbartonshire',
 'East Lothian',
 'East Sussex',
 'East Yorkshire',
 'Edinburgh',
 'Essex',
 'Falkirk',
 'Fife',
 'Flintshire',
 'Glasgow',
 'Gloucestershire',
 'Gwynedd',
 'Hampshire',
 'Herefordshire',
 'Hertfordshire',
 'Highland',
 'Inverclyde',
 'Isle of Wight',
 'Kent',
 'Lancashire',
 'Leicestershire',
 'Lincolnshire',
 'London',
 'Manchester',
 'Merseyside',
 'Merthyr Tydfil',
 'Midlothian',
 'Monmouthshire',
 'Moray',
 'Neath Port Talbot',
 'Newport',
 'Norfolk',


Finally I'll check for any duplicated data.

In [ ]:
#TODO Identify duplicate data